# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
tsch = len(school_data_complete['school_name'].unique())
tst = school_data_complete['student_name'].count()
tb = school_data['budget'].sum()
avm = student_data['math_score'].mean()
avr = student_data['reading_score'].mean()

mpass = student_data['math_score'] >= 70
rpass = student_data['reading_score'] >= 70

mpr = (mpass.value_counts().array[0]/mpass.count())*100
rpr = (rpass.value_counts().array[0]/rpass.count())*100
opr = ((mpr + rpr)/2)

d = {
    'tsch':[tsch],'ts':[tst],'tb':[tb],'avm':[avm],
    'avr':[avr],'mpr':[mpr],'rpr':[rpr],'opr':[opr]
}

district_summary = pd.DataFrame(data = d)

district_summary['tb'] = district_summary['tb'].map('${:}'.format)
district_summary['avm'] = district_summary['avm'].map('{:.2f}'.format)
district_summary['avr'] = district_summary['avr'].map('{:.2f}'.format)
district_summary['rpr'] = district_summary['rpr'].map('{:.2f}%'.format)
district_summary['opr'] = district_summary['opr'].map('{:.2f}%'.format)
district_summary['mpr'] = district_summary['mpr'].map('{:.2f}%'.format)


column_titles = ['Total Schools','Total Students','Total Budget',
                 'Average Math Score','Average Reading Score','% Passing Math',
                 '% Passing Reading','% Overall Passing Rate']
count = 0
j = district_summary.columns
for i in j:
    district_summary = district_summary.rename(
        columns={i:column_titles[count]})
    count += 1

district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,$24649428,78.99,81.88,74.98%,85.81%,80.39%


In [3]:
def passing(i):
    ipass = i >= 70
    return (ipass.value_counts().array[0]/ipass.count())*100

In [4]:
school_group = school_data_complete.groupby('school_name')

school_aggregation = school_group.agg({
    'reading_score': ['mean',passing],
    'math_score':['mean',passing]
})

In [5]:
psb = school_data['budget'].div(school_data['size'])

type(psb)

pandas.core.series.Series

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [6]:
psb = school_data['budget'].div(school_data['size'])
avm = school_aggregation['math_score']['mean']
avr = school_aggregation['reading_score']['mean']
ppm = school_aggregation['math_score']['passing']
ppr = school_aggregation['math_score']['passing']
opr = (ppm.add(ppr))/2

school_summary = school_data
school_summary['psb'] = psb
school_summary.rename(columns = {'school_name':'School Name'},inplace = True)
school_summary = school_summary.drop(columns = 'School ID')
school_summary = school_summary.set_index('School Name')
school_summary['avm'] = avm
school_summary['avr'] = avr
school_summary['ppm'] = ppm
school_summary['ppr'] = ppr
school_summary['opr'] = opr

column_titles = ['School Type','Total Students','Total School Budget',
                 'Per Student Budget','Average Math Score',
                 'Average Reading Score','% Passing Math',
                 '% Passing Reading','% Passing Overall']
count = 0
j = school_summary.columns
for i in j:
    school_summary = school_summary.rename(
        columns={i:column_titles[count]})
    count += 1
#school_summary.head()

In [7]:
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,65.683922,65.683922
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,65.988471,65.988471
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,93.867121,93.867121
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,66.752967,66.752967
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,93.392371,93.392371


## Top Performing Schools (By Passing Rate)

In [8]:
school_summary_top = school_summary.sort_values(by=['% Passing Overall'],
                                                ascending = False)
school_summary_top.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,94.594595,94.594595
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,94.133477,94.133477
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,93.867718,93.867718
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,93.867121,93.867121
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,93.392371,93.392371


* Sort and display the top five schools in overall passing rate

## Bottom Performing Schools (By Passing Rate)

In [9]:
school_summary_bottom = school_summary.sort_values(by=['% Passing Overall'])
school_summary_bottom.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,65.683922,65.683922
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,65.988471,65.988471
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,66.057551,66.057551
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,66.366592,66.366592
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,66.680064,66.680064


* Sort and display the five worst-performing schools

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
math_by_grade = school_data_complete[['school_name','math_score']]

nine = school_data_complete['grade'] == '9th'
ten = school_data_complete['grade'] == '10th'
eleven = school_data_complete['grade'] == '11th'
twelve = school_data_complete['grade'] == '12th'

loc9 = math_by_grade.loc[nine,:]
loc10 = math_by_grade.loc[ten,:]
loc11 = math_by_grade.loc[eleven,:]
loc12 = math_by_grade.loc[twelve,:]

math_scores = pd.DataFrame(loc9.groupby('school_name').mean())
math_scores.rename(columns={'math_score':'9th'}, inplace = True)
math_scores['10th'] = loc10.groupby('school_name').mean()
math_scores['11th'] = loc11.groupby('school_name').mean()
math_scores['12th'] = loc12.groupby('school_name').mean()

math_scores.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [14]:
reading_by_grade = school_data_complete[['school_name','reading_score']]

nine = school_data_complete['grade'] == '9th'
ten = school_data_complete['grade'] == '10th'
eleven = school_data_complete['grade'] == '11th'
twelve = school_data_complete['grade'] == '12th'

loc9 = reading_by_grade.loc[nine,:]
loc10 = reading_by_grade.loc[ten,:]
loc11 = reading_by_grade.loc[eleven,:]
loc12 = reading_by_grade.loc[twelve,:]

reading_scores = pd.DataFrame(loc9.groupby('school_name').mean())
reading_scores.rename(columns={'reading_score':'9th'}, inplace = True)
reading_scores['10th'] = loc10.groupby('school_name').mean()
reading_scores['11th'] = loc11.groupby('school_name').mean()
reading_scores['12th'] = loc12.groupby('school_name').mean()

reading_scores.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [16]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [21]:
school_summary['Per Student Spending Ranges'] = pd.cut(school_summary['Per Student Budget'],spending_bins,labels=group_names)

ssg = school_summary.groupby('Per Student Spending Ranges')

ssg.agg({
    'Average Math Score':['mean'],
    'Average Reading Score':['mean'],
    '% Passing Math':['mean'],
    '% Passing Reading':['mean'],
    '% Passing Overall':['mean']
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,mean,mean,mean,mean,mean
Per Student Spending Ranges,,,,,
<$585,83.455399,83.933814,93.460096,93.460096,93.460096
$585-615,83.599686,83.885211,94.230858,94.230858,94.230858
$615-645,79.079225,81.891436,75.668212,75.668212,75.668212
$645-675,76.997210,81.027843,66.164813,66.164813,66.164813


## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [27]:
school_summary['School Size'] = pd.cut(school_summary['Total Students'],size_bins,labels=group_names)

ssg = school_summary.groupby('School Size')

ssg.agg({
    'Average Math Score':['mean'],
    'Average Reading Score':['mean'],
    '% Passing Math':['mean'],
    '% Passing Reading':['mean'],
    '% Passing Overall':['mean']
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,mean,mean,mean,mean,mean
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,93.550225,93.550225
Medium (1000-2000),83.374684,83.864438,93.599695,93.599695,93.599695
Large (2000-5000),77.746417,81.344493,69.963361,69.963361,69.963361


## Scores by School Type

* Perform the same operations as above, based on school type.

In [28]:


ssg = school_summary.groupby('School Type')

ssg.agg({
    'Average Math Score':['mean'],
    'Average Reading Score':['mean'],
    '% Passing Math':['mean'],
    '% Passing Reading':['mean'],
    '% Passing Overall':['mean']
})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,mean,mean,mean,mean,mean
School Type,,,,,
Charter,83.473852,83.896421,93.620830,93.620830,93.620830
District,76.956733,80.966636,66.548453,66.548453,66.548453
